In [1]:
from data import read_csv

In [2]:
train_path = "..//data//train.csv"
test_path = "..//data//test.csv"

In [3]:
header,train = read_csv(train_path)
_,test = read_csv(test_path,test=True)
print(test[0])

['ID58593', 0.341731678878033, 0.0, 0.586538461538462, 0, 4076, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 796, 3, 0, 5, 0, 4.6, 3445, 1515, 1475, 1185, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
class NaiveBayesClassifier:
    def __init__(self):
        print("NaiveBayesClassfier is created")
        
    def fit(self,data):
        self.data = data
        self.categorical_fit()
    
    def categorical_fit(self):
        self.cat = [4,7,8,9,10,11,12,14,15,16,17,18,19,22,24,30,31,32,33,34,35,36,37,38,39,40,41]
        self.posterior = dict()
        
        # Initialize
        for c in self.cat:
            self.posterior[c] = [0,0]
        
        for d in self.data:
            claim = d[43]
            for c in self.cat:
                self.posterior[c][claim] += 1
        
        for k,v in self.posterior.items():
            print(k,v)

In [27]:
clf = NaiveBayesClassifier()
clf.fit(train)

NaiveBayesClassfier is created
4 [54844, 3748]
7 [54844, 3748]
8 [54844, 3748]
9 [54844, 3748]
10 [54844, 3748]
11 [54844, 3748]
12 [54844, 3748]
14 [54844, 3748]
15 [54844, 3748]
16 [54844, 3748]
17 [54844, 3748]
18 [54844, 3748]
19 [54844, 3748]
22 [54844, 3748]
24 [54844, 3748]
30 [54844, 3748]
31 [54844, 3748]
32 [54844, 3748]
33 [54844, 3748]
34 [54844, 3748]
35 [54844, 3748]
36 [54844, 3748]
37 [54844, 3748]
38 [54844, 3748]
39 [54844, 3748]
40 [54844, 3748]
41 [54844, 3748]


In [15]:
train[:2][:-1]

[['ID00001',
  0.515873589958172,
  0.05,
  0.644230769230769,
  0,
  4990,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  796,
  3,
  0,
  5,
  0,
  4.6,
  3445,
  1515,
  1475,
  1185,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]